[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/moustakas/ENGR320/blob/main/foundations.ipynb)

# ENGR 320-Spring 2026
## Computational Homework 1: Foundations

---

This assignment reviews numerical differentiation and integration techniques that we'll use throughout the course. By the end, you should be comfortable:

- Computing derivatives from discrete data using `np.gradient`
- Computing integrals using `np.trapezoid` and `scipy.integrate.quad`
- Visualizing results with `matplotlib`

## Part 0: Setup

We'll use three libraries throughout this course:

- `numpy` — arrays, math functions, `linspace`, `gradient`, `trapz`
- `matplotlib.pyplot` — plotting
- `scipy.integrate` — numerical integration (`quad`)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

---

## Part 1: Numerical Differentiation — Fourier's Law

Fourier's law relates the heat flux $q$ to the temperature gradient:

$$q = -k \frac{dT}{dx}$$

where $k$ is the thermal conductivity. Given a temperature profile $T(x)$, we can compute the heat flux by numerically differentiating.

### Example: Nonlinear temperature profile

Consider a wall of thickness $L$ with a quadratic temperature profile:

$$T(x) = T_1 + (T_2 - T_1)\left(\frac{x}{L}\right)^2$$

This might arise from internal heat generation. The analytical derivative is:

$$\frac{dT}{dx} = \frac{2(T_2 - T_1)}{L^2} x$$

### Step 1: Define parameters and sample the temperature profile

In [ ]:
# Parameters
L = 0.1       # wall thickness, m
T1 = 300.0    # temperature at x=0, K
T2 = 400.0    # temperature at x=L, K

In [ ]:
# Create spatial grid
N = 50
x = np.linspace(0, L, N)  # m

In [ ]:
# Compute temperature at each grid point
T = T1 + (T2 - T1) * (x / L)**2  # K

In [ ]:
# Plot the temperature profile
plt.figure()
plt.plot(x * 100, T)  # convert x to cm for readability
plt.xlabel('Position (cm)')
plt.ylabel('Temperature (K)')
plt.title('Temperature Profile in Wall')
plt.grid(True)
plt.show()

### Step 2: Compute the temperature gradient numerically

`np.gradient(y, x)` computes the derivative $dy/dx$ using central differences (and one-sided differences at the boundaries).

In [ ]:
# Numerical derivative
dTdx_numerical = np.gradient(T, x)  # K/m

In [ ]:
# Analytical derivative for comparison
dTdx_analytical = 2 * (T2 - T1) / L**2 * x  # K/m

In [ ]:
# Compare numerical and analytical derivatives
plt.figure()
plt.plot(x * 100, dTdx_numerical, 'b-', label='Numerical')
plt.plot(x * 100, dTdx_analytical, 'r--', label='Analytical')
plt.xlabel('Position (cm)')
plt.ylabel('dT/dx (K/m)')
plt.title('Temperature Gradient: Numerical vs. Analytical')
plt.legend()
plt.grid(True)
plt.show()

The numerical and analytical results should nearly overlap. Small differences at the boundaries arise because `np.gradient` uses one-sided differences there.

### Step 3: Compute heat flux for different thermal conductivities

Using Fourier's law, $q = -k \, dT/dx$:

In [ ]:
# Thermal conductivities to compare (W/(m·K))
k_values = [1, 10, 50]

plt.figure()
for k in k_values:
    q = -k * dTdx_numerical  # W/m^2
    plt.plot(x * 100, q / 1000, label=f'k = {k} W/(m·K)')  # convert to kW/m^2

plt.xlabel('Position (cm)')
plt.ylabel('Heat Flux (kW/m²)')
plt.title('Heat Flux for Different Thermal Conductivities')
plt.legend()
plt.grid(True)
plt.show()

Higher thermal conductivity produces a larger heat flux for the same temperature gradient.

---

## Part 2: Numerical Integration — Pipe Flow

For fully developed laminar flow in a circular pipe, the velocity profile is parabolic:

$$u(r) = u_{\max}\left(1 - \frac{r^2}{R^2}\right)$$

where $R$ is the pipe radius and $u_{\max}$ is the centerline velocity.

The volumetric flow rate is found by integrating over the cross-section:

$$Q = \int_0^R u(r) \cdot 2\pi r \, dr$$

The analytical result is:

$$Q = \frac{\pi}{2} u_{\max} R^2$$

### Step 1: Define the velocity profile

In [ ]:
def velocity(r, u_max, R):
    """Laminar pipe flow velocity profile.
    
    Parameters
    ----------
    r : float or array
        Radial position (m)
    u_max : float
        Centerline velocity (m/s)
    R : float
        Pipe radius (m)
    
    Returns
    -------
    u : float or array
        Velocity at position r (m/s)
    """
    return u_max * (1 - (r / R)**2)

### Step 2: Plot velocity profiles for different pipe radii

In [ ]:
u_max = 2.0  # centerline velocity, m/s
R_values = [0.01, 0.02, 0.05]  # pipe radii, m

plt.figure()
for R in R_values:
    r = np.linspace(0, R, 100)
    u = velocity(r, u_max, R)
    plt.plot(r * 100, u / u_max, label=f'R = {R*100:.0f} cm')

plt.xlabel('Radial Position (cm)')
plt.ylabel(r'$u / u_{max}$')
plt.title('Normalized Velocity Profiles')
plt.legend()
plt.grid(True)
plt.show()

### Step 3: Compute volumetric flow rate — Trapezoidal rule

First, we'll sample the integrand $f(r) = 2\pi r \, u(r)$ and use `np.trapezoid`:

In [ ]:
R = 0.02  # m
N = 100
r = np.linspace(0, R, N)

In [ ]:
# Integrand: 2*pi*r*u(r)
u = velocity(r, u_max, R)
integrand = 2 * np.pi * r * u

In [ ]:
# Trapezoidal integration
Q_trapz = np.trapezoid(integrand, r)  # m^3/s

print(f'Trapezoidal rule: Q = {Q_trapz:.6e} m³/s')

### Step 4: Compute volumetric flow rate — `scipy.integrate.quad`

`quad` performs adaptive quadrature on a continuous function, generally giving higher accuracy:

In [ ]:
def integrand_func(r, u_max, R):
    """Integrand for volumetric flow rate: 2*pi*r*u(r)"""
    return 2 * np.pi * r * velocity(r, u_max, R)

Q_quad, error = integrate.quad(integrand_func, 0, R, args=(u_max, R))

print(f'scipy.integrate.quad: Q = {Q_quad:.6e} m³/s')

### Step 5: Compare to analytical result

In [ ]:
Q_analytical = (np.pi / 2) * u_max * R**2

In [ ]:
print(f'Analytical:          Q = {Q_analytical:.6e} m³/s')
print(f'\nTrapezoidal error: {abs(Q_trapz - Q_analytical) / Q_analytical * 100:.4f}%')
print(f'Quad error:        {abs(Q_quad - Q_analytical) / Q_analytical * 100:.4f}%')

Both methods should give results very close to the analytical value. The trapezoidal rule accuracy depends on the number of grid points; `quad` is adaptive and typically more accurate for smooth functions.

---

## Part 3: Exercises

Complete the following exercises. Show your code and results.

### Exercise 1: Temperature gradient from measured data

The arrays below contain temperature measurements along a rod. Compute and plot the temperature gradient $dT/dx$.

In [ ]:
# Measured data
x_data = np.array([0.0, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20])  # m
T_data = np.array([350.0, 342.1, 331.5, 318.2, 302.4, 285.0, 267.3, 251.2, 238.6, 231.1, 229.0])  # K

In [ ]:
# Your code here:


### Exercise 2: Flow rate from velocity measurements

The arrays below contain velocity measurements across a pipe cross-section. Compute the volumetric flow rate using `np.trapezoid`.

In [ ]:
# Measured data (pipe radius R = 0.025 m)
r_data = np.array([0.0, 0.005, 0.010, 0.015, 0.020, 0.025])  # m
u_data = np.array([1.82, 1.75, 1.54, 1.19, 0.70, 0.0])       # m/s

In [ ]:
# Your code here:


### Exercise 3 (Bonus): Effect of grid resolution on integration accuracy

Using the laminar pipe flow example from Part 2 (with $R = 0.02$ m and $u_{\max} = 2$ m/s), investigate how the trapezoidal rule accuracy depends on the number of grid points.

1. Compute $Q$ using `np.trapezoid` for $N = 5, 10, 20, 50, 100, 200$ grid points
2. Compute the percent error relative to the analytical solution for each case
3. Plot the percent error vs. $N$ (hint: a log-log plot may be informative)

In [ ]:
# Your code here:
